In [1]:
import os
from gensim.utils import simple_preprocess

# List to store sentences
all_sentences = []

# Loop through files named '001ssb.txt' to '005ssb.txt'
for i in range(1, 6):
    # Create the filename
    filename = f'dataset/{i:03d}ssb.txt'
    
    # Open the file and read the contents with error handling
    try:
        with open(filename, 'r', encoding='utf-8', errors='replace') as file:
            # Read the file content
            content = file.read()
            
            # Split the content into sentences (can be further customized as needed)
            sentences = content.split('.')  # Assuming sentences are separated by periods
            
            # Tokenize each sentence using simple_preprocess
            tokenized_sentences = [simple_preprocess(sentence) for sentence in sentences if sentence]
            
            # Add the tokenized sentences to the list
            all_sentences.extend(tokenized_sentences)
    
    except UnicodeDecodeError as e:
        print(f"Error reading file {filename}: {e}")

# Output the sentences for verification
print(len(all_sentences))


146453


In [2]:
for sentence in all_sentences[:3]:
    print(sentence)

['game', 'of', 'thrones', 'book', 'one', 'of', 'song', 'of', 'ice', 'and', 'fire', 'by', 'george']
[]
['martin', 'prologue', 'we', 'should', 'start', 'back', 'gared', 'urged', 'as', 'the', 'woods', 'began', 'to', 'grow', 'dark', 'around', 'them']


In [3]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

# Remove stopwords from tokenized sentences
cleaned_sentences = [[word for word in sentence if word not in STOPWORDS] for sentence in all_sentences]
len(cleaned_sentences)

146453

In [4]:
for sentence in cleaned_sentences[:3]:
    print(sentence)

['game', 'thrones', 'book', 'song', 'ice', 'george']
[]
['martin', 'prologue', 'start', 'gared', 'urged', 'woods', 'began', 'grow', 'dark']


In [5]:
cleaned_sentences = [
    ['iron_throne' if word == 'iron' and idx + 1 < len(sentence) and sentence[idx + 1] == 'throne' 
     else word for idx, word in enumerate(sentence)] 
    for sentence in cleaned_sentences
]
len(cleaned_sentences)

146453

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from itertools import chain
import numpy as np
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

# Step 1: Tokenization
# Build vocabulary and map words to indices
def build_vocab(sentences):
    all_words = list(chain.from_iterable(sentences))
    word_counts = Counter(all_words)
    vocab = {word: idx + 2 for idx, word in enumerate(word_counts)}  # +2 for padding and unknown
    vocab['<PAD>'] = 0
    vocab['<UNK>'] = 1
    return vocab

def tokenize_sentence(sentence, vocab):
    return [vocab.get(word, vocab['<UNK>']) for word in sentence]

# Step 2: Create Dataset
class GOTDataset(Dataset):
    def __init__(self, sentences, vocab):
        self.sentences = [tokenize_sentence(sentence, vocab) for sentence in sentences]
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        return torch.tensor(self.sentences[idx], dtype=torch.long)

# Step 3: LSTM Model Definition
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, (hidden, cell) = self.lstm(embedded)
        output = self.fc(hidden[-1])  # Use the last hidden state for classification
        return output.view(-1, 1)  # Reshape to (batch_size, 1)

# Step 4: Padding
def collate_fn(batch):
    # Pad the sequences to the length of the longest sequence in the batch
    batch = pad_sequence(batch, batch_first=True, padding_value=0)
    return batch

# Prepare data
vocab = build_vocab(cleaned_sentences)
dataset = GOTDataset(cleaned_sentences, vocab)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Step 5: Train the model
vocab_size = len(vocab)
embedding_dim = 100  # Same as Word2Vec vector size
hidden_dim = 128
output_dim = 1  # You can modify this based on your task
num_layers = 2

model = LSTMModel(vocab_size, embedding_dim, hidden_dim, output_dim, num_layers)

# Loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # For binary classification, change to CrossEntropyLoss for multi-class
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(batch)
        
        # Assuming you have binary classification labels (0s and 1s)
        labels = torch.ones_like(outputs)  # Replace this with your actual labels
        labels = labels.view(-1, 1)  # Ensure labels have shape (batch_size, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}')
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}')


Epoch 1/10, Loss: 0.0014816516796277094
Epoch 1/10, Loss: 0.0014816516796277094
Epoch 2/10, Loss: 4.796326365560414e-07
Epoch 2/10, Loss: 4.796326365560414e-07
Epoch 3/10, Loss: 6.54142245294683e-08
Epoch 3/10, Loss: 6.54142245294683e-08
Epoch 4/10, Loss: 5.639256717651377e-08
Epoch 4/10, Loss: 5.639256717651377e-08
Epoch 5/10, Loss: 5.2225865847682225e-08
Epoch 5/10, Loss: 5.2225865847682225e-08
Epoch 6/10, Loss: 4.277900221733462e-08
Epoch 6/10, Loss: 4.277900221733462e-08
Epoch 7/10, Loss: 3.727926948139908e-08
Epoch 7/10, Loss: 3.727926948139908e-08
Epoch 8/10, Loss: 3.7280028312165856e-08
Epoch 8/10, Loss: 3.7280028312165856e-08
Epoch 9/10, Loss: 3.7283561960571904e-08
Epoch 9/10, Loss: 3.7283561960571904e-08
Epoch 10/10, Loss: 3.688039127838545e-08
Epoch 10/10, Loss: 3.688039127838545e-08


In [9]:
torch.save(model, 'lstm_model.pth')

In [30]:
print(len(model.embedding.weight.data))

IndexError: too many indices for tensor of dimension 2

In [22]:
import torch
import torch.nn.functional as F
import pandas as pd

# Assuming 'model' is your trained LSTM model with an embedding layer
# Create a vocabulary mapping from words to indices
# This assumes 'model.embedding' is your embedding layer
vocab_mapping = {word: idx for idx, word in enumerate(model.embedding.weight.data)}
# Function to get embedding vector from the LSTM model
def get_embedding(name):
    # Convert name to index
    name_idx = vocab_mapping.get(name)
    print(name)
    if name_idx is not None:  # Ensure the name exists in the vocabulary
        input_tensor = torch.tensor([[name_idx]])  # Shape: (1, 1)
        with torch.no_grad():
            embedding_vector = model.embedding(input_tensor)
        return embedding_vector.squeeze(0).squeeze(0)  # Shape: (embedding_dim)
    return None  # Return None if the name is not found

# Function to calculate similarity considering both first and last name using LSTM model
def compute_similarity(name):
    name_parts = name.split(" ")
    first_name = name_parts[0].lower()
    last_name = name_parts[1].lower() if len(name_parts) > 1 else None
    
    similarities = []
    
    # Calculate similarity for the first name
    first_name_embedding = get_embedding(first_name)
    print(first_name_embedding)
    if first_name_embedding is not None:
        iron_throne_embedding = get_embedding('iron_throne')
        if iron_throne_embedding is not None:
            similarity = F.cosine_similarity(first_name_embedding, iron_throne_embedding, dim=0)
            similarities.append(similarity.item())  # Convert tensor to float
    # Calculate similarity for the last name (if it exists)
    if last_name:
        last_name_embedding = get_embedding(last_name)
        if last_name_embedding is not None:
            iron_throne_embedding = get_embedding('iron_throne')
            if iron_throne_embedding is not None:
                similarity = F.cosine_similarity(last_name_embedding, iron_throne_embedding, dim=0)
                similarities.append(similarity.item())  # Convert tensor to float
    
    # Return the average similarity if both names exist, otherwise just one similarity
    if similarities:
        return sum(similarities) / len(similarities)
    else:
        return None  # Return None if no name part is in the vocabulary

# Function to get last name
def get_last_name(name):
    name_parts = name.split(" ")
    last_name = name_parts[1].lower() if len(name_parts) > 1 else None
    return last_name
        
# Sample DataFrame with character names (replace this with your actual DataFrame)
characters = pd.DataFrame({'Name': ['Jon Snow', 'Daenerys Targaryen', 'Tyrion Lannister', 'Cersei Lannister']})

# Apply the function to calculate similarity for each character
characters['similarity'] = characters['Name'].apply(compute_similarity)

# Apply the function to get last names for each character
characters['house'] = characters['Name'].apply(get_last_name)

# Print the resulting DataFrame
print(characters.head())


jon
None
snow
daenerys
None
targaryen
tyrion
None
lannister
cersei
None
lannister
                 Name similarity      house
0            Jon Snow       None       snow
1  Daenerys Targaryen       None  targaryen
2    Tyrion Lannister       None  lannister
3    Cersei Lannister       None  lannister


In [21]:
model_cbow.wv['jon']

array([ 7.5620943e-01, -2.4885155e-01,  1.1386960e+00,  3.7612301e-01,
       -2.6545791e-02, -1.4147528e+00,  8.2844299e-01, -4.3518519e-01,
       -5.9370720e-01, -9.5779550e-01,  2.4015382e-01, -6.1506528e-01,
        9.2488825e-03,  1.1070847e-01, -4.3834284e-01, -4.4569808e-01,
       -3.6657352e-02, -2.8070518e-01, -9.3288469e-01, -1.0664799e+00,
        1.7675240e-01,  7.1801430e-01,  5.5871081e-01, -1.3113753e+00,
       -4.6027428e-01,  9.0649945e-01, -8.9804769e-01,  7.7424628e-01,
       -8.0078834e-01,  6.5633863e-01, -2.1794173e-01,  2.5627020e-01,
        5.9267098e-01, -1.7264159e+00, -8.7621361e-02,  1.1811959e-01,
       -4.0051216e-01,  3.7502930e-01, -4.6692187e-01,  1.3635179e-01,
        8.6818302e-01, -2.6004326e-01, -5.4952836e-01,  2.5428715e-01,
       -7.0807226e-02, -1.3207475e+00, -7.0047897e-01, -2.3358734e-01,
        8.0037642e-01,  8.6103618e-01, -3.6515000e-01, -6.9662023e-01,
        1.0320263e+00, -8.9662910e-01, -4.1205952e-01,  1.3783135e+00,
      

In [22]:
vector_king_cbow

array([-3.3909363e-01, -2.2002006e-01,  4.7506067e-01,  7.7871668e-01,
        5.3651970e-02, -5.2936649e-01,  1.5886328e+00,  2.0001726e-01,
       -1.2297843e+00,  9.4697887e-01, -1.5030047e-01, -9.2730635e-01,
        3.9230552e-02,  4.4592194e-02,  3.9217886e-02, -1.3617882e+00,
        8.2519382e-02, -4.6770964e-02, -6.2201005e-01, -1.3790336e+00,
        8.3843723e-02, -9.9067819e-01,  1.0594131e+00,  2.1284914e-01,
       -7.4863583e-02,  6.1514056e-01, -6.4188844e-01, -6.2276477e-01,
       -1.7749134e-01,  2.2741379e-01, -6.9773555e-02, -5.3160846e-01,
        6.9203397e-04,  7.8685653e-01,  1.2047927e+00,  1.0464323e+00,
       -3.2411852e-01,  6.0192398e-03, -1.3213059e+00, -1.3241209e+00,
        5.8795843e-02, -1.0450821e+00, -1.1407553e+00,  2.7123210e-01,
        4.2258427e-01, -6.0977143e-01, -3.1508815e-01,  4.4198966e-01,
        8.4960675e-01,  6.7589730e-01,  2.7892965e-01, -5.4389244e-01,
       -9.2235573e-02,  6.5729505e-01, -1.1411744e+00, -3.4988809e-01,
      

In [26]:
# Find most similar words to 'king' in the CBOW model
similar_words_cbow = model_cbow.wv.most_similar(positive=[vector_king_cbow], topn=10)

# Find most similar words to 'king' in the Skip-Gram model
similar_words_skipgram = model_skipgram.wv.most_similar(positive=[vector_king_skipgram], topn=10)

# Print the similar words along with their similarity scores
#print("Similar words to 'iron throne' in CBOW model:", similar_words_cbow)
#print("Similar words to 'iron throne' in Skip-Gram model:", similar_words_skipgram)

In [96]:
similarity_score = model_cbow.wv.similarity('iron_throne', 'stark')
similarity_score

0.381342

In [53]:
import pandas as pd

characters = pd.read_csv('dataset/character-deaths.csv')[["Name"]]
characters.head()

,Name
0,Addam Marbrand
1,Aegon Frey (Jinglebell)
2,Aegon Targaryen
3,Adrack Humble
4,Aemon Costayne


In [89]:
characters['Name'].iloc[0].split(" ")[1].lower()

'marbrand'

In [91]:
# Function to calculate similarity considering both first and last name
def compute_similarity(name):
    name_parts = name.split(" ")
    first_name = name_parts[0].lower()
    last_name = name_parts[1].lower() if len(name_parts) > 1 else None
    
    similarities = []
    
    # Calculate similarity for the first name
    if first_name in model_cbow.wv:
        similarities.append(model_cbow.wv.similarity('iron_throne', first_name))
    
    # Calculate similarity for the last name (if it exists)
    if last_name and last_name in model_cbow.wv:
        similarities.append(model_cbow.wv.similarity('iron_throne', last_name))
    
    # Return the average similarity if both names exist, otherwise just one similarity
    if similarities:
        return sum(similarities) / len(similarities)
    else:
        return None  # Return None if no name part is in the vocabulary

# Function to calculate similarity considering both first and last name
def get_last_name(name):
    name_parts = name.split(" ")
    last_name = name_parts[1].lower() if len(name_parts) > 1 else None
    return last_name
        
# Apply the function to calculate similarity for each character
characters['similarity'] = characters['Name'].apply(compute_similarity)

# Apply the function to calculate similarity for each character
characters['house'] = characters['Name'].apply(get_last_name)

# Print the resulting DataFrame
print(characters.head())


                      Name  similarity      house
0           Addam Marbrand    0.464045   marbrand
1  Aegon Frey (Jinglebell)    0.613120       frey
2          Aegon Targaryen    0.744675  targaryen
3            Adrack Humble    0.561879     humble
4           Aemon Costayne    0.492332   costayne


In [82]:
characters[:4]

,Name,similarity
0,Addam Marbrand,0.464045
1,Aegon Frey (Jinglebell),0.613120
2,Aegon Targaryen,0.744675
3,Adrack Humble,0.561879


In [92]:
# Sort the DataFrame by 'similarity' column in descending order
characters_sorted = characters.sort_values(by='similarity', ascending=False)

# Print the sorted DataFrame

characters_sorted


,Name,similarity,house
2,Aegon Targaryen,0.744675,targaryen
689,Renly Baratheon,0.717126,baratheon
702,Robert Baratheon,0.716023,baratheon
411,Joffrey Baratheon,0.712842,baratheon
911,Zei,0.707267,None
...,...,...,...
379,Jack-Be-Lucky,NaN,None
405,Jommy,NaN,None
805,Three-Tooth,NaN,None
881,Will,NaN,None


In [93]:
characters_sorted.to_csv('sorted.csv')